In [2]:
import os
os.chdir("src")

In [15]:
from types import SimpleNamespace
from utils import *
from moe_utils import *
from moe_architecture import *

In [16]:
args = SimpleNamespace()
args.lora_rank=2
args.lora_architecture="moe"
args.num_post_train_epochs=1
args.learning_rate=3e-4
args.model_name="qwen2.5-1.5b-instruct"
args.learning_rate=3e-4

In [17]:
model, tokenizer, config = get_model(model_name="qwen2.5-1.5b-instruct")

In [18]:
trained_adapter_dir = f"/scratch/doluk/Compact-Interference-PRAG/offline/{args.model_name}/rank={args.lora_rank}_alpha=32/popqa/lr=0.0003_epoch=2_direct/aug_model=qwen2.5-1.5b-instruct/total/data_0"  # <-- set your path
# 2) Re-inject adapters (loads frozen A/B from trained_adapter_dir)
inject_hydra_lora(
    model,
    trained_data_adapters_dir=trained_adapter_dir,
    r=args.lora_rank,
    alpha=32,
    target_modules=TARGET_MODULES,
    architecture=args.lora_architecture,
)

In [19]:
model.to(device=DEVICE)
model.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): LinearWithCustomLoRA(
            (base): Linear(in_features=1536, out_features=8960, bias=False)
            (custom_lora): MoELoRA(
              (As): ModuleList(
                (0-2): 3 x Linear(in_features=1536, out_features=2, bias=False)
              )
              (Bs): ModuleList(
                (0-2): 3 x Linear(in_features=2, out_features=8960, bias=False)
              )
              (route

In [24]:
from safetensors.torch import load_file
load_dir=os.path.join(TESTING_SAVE_PATH, args.lora_architecture,  f"lr={args.learning_rate}_epoch={args.num_post_train_epochs}")
os.path.exists(load_dir)
path = os.path.join(load_dir, "adapter_model.safetensors")
sd = load_file(path)

In [27]:
incompatible = model.load_state_dict(sd, strict=False)

In [39]:
list(sd.keys())

['model.layers.0.mlp.down_proj.custom_lora.router.bias',
 'model.layers.0.mlp.down_proj.custom_lora.router.weight',
 'model.layers.0.mlp.gate_proj.custom_lora.router.bias',
 'model.layers.0.mlp.gate_proj.custom_lora.router.weight',
 'model.layers.0.mlp.up_proj.custom_lora.router.bias',
 'model.layers.0.mlp.up_proj.custom_lora.router.weight',
 'model.layers.1.mlp.down_proj.custom_lora.router.bias',
 'model.layers.1.mlp.down_proj.custom_lora.router.weight',
 'model.layers.1.mlp.gate_proj.custom_lora.router.bias',
 'model.layers.1.mlp.gate_proj.custom_lora.router.weight',
 'model.layers.1.mlp.up_proj.custom_lora.router.bias',
 'model.layers.1.mlp.up_proj.custom_lora.router.weight',
 'model.layers.10.mlp.down_proj.custom_lora.router.bias',
 'model.layers.10.mlp.down_proj.custom_lora.router.weight',
 'model.layers.10.mlp.gate_proj.custom_lora.router.bias',
 'model.layers.10.mlp.gate_proj.custom_lora.router.weight',
 'model.layers.10.mlp.up_proj.custom_lora.router.bias',
 'model.layers.10.ml

In [40]:
for key in incompatible.missing_keys:
    if "model.layers" in key and "router" in key:
        print(key)

In [ ]:
model, tokenizer, config = get_model(model_name="qwen2.5-1.5b-instruct")

# 2) Re-inject adapters (loads frozen A/B from trained_adapter_dir)
inject_hydra_lora(
    model,
    trained_data_adapters_dir=trained_adapter_dir,
    r=args.lora_rank,
    alpha=32,
    target_modules=TARGET_MODULES,
    architecture=args.lora_architecture,
)
model.to(device=DEVICE)
model.eval()

# 3) Load the trained router/mixture weights
load_injected_adapters(model, os.path.join(TESTING_SAVE_PATH, args.lora_architecture,  f"lr={args.learning_rate}_epoch={args.num_post_train_epochs}"), strict=False)

# 4) Run inference
with torch.no_grad():
    print(model_generate("What is George Rankin's occupation?", model, tokenizer, config))
    